In [1]:
require(dplyr)
require(ggpubr)
require(data.table)
library(tidyverse)
options(warn = -1)

Loading required package: dplyr


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: ggpubr

Loading required package: ggplot2

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


── Attaching core tidyverse packages ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
✔ readr     2.1.4     
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_con

**Parameters**

In [2]:
base_path <- "/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/07_gene_regulatory_network/results"

In [3]:
# Inputs
inputpath_perez2022_level1 <- "data/STAT1_SP1_perez2022_level1.csv"
inputpath_scgt00_level1 <- "data/STAT1_SP1_scgt00_level1.csv"
inputpath_perez2022 <- "data/STAT1_SP1_perez2022.csv"
inputpath_scgt00 <- "data/STAT1_SP1_scgt00.csv"
inputpath_sle_metadata <- "/scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas/03_downstream_analysis/06_inflammation_signatures/results/figures/IFN_response_analysis/SLE_metadata.csv"

# Palette
color_palette_path <- "data/colors_palette.R"

# Outputs
outputpath <- "results/"

# Load data

In [4]:
perez2022_l1 <- read.csv(inputpath_perez2022_level1, header = TRUE, row.names = 1)
scgt00_l1 <- read.csv(inputpath_scgt00_level1, header = TRUE, row.names = 1)
perez2022 <- read.csv(inputpath_perez2022, header = TRUE, row.names = 1)
scgt00 <- read.csv(inputpath_scgt00, header = TRUE, row.names = 1)
source(color_palette_path)

In [5]:
SLE_metadata = read.csv(inputpath_sle_metadata, header = TRUE, row.names = 1)
print(head(SLE_metadata))

                  SLEDAI_score Flare Responder   studyID
Perez2022_1004_T0            2  notF           Perez2022
Perez2022_1014_T0            2  notF           Perez2022
Perez2022_1019_T0            0  notF           Perez2022
Perez2022_1022_T0            2  notF           Perez2022
Perez2022_1031_T0            2  notF           Perez2022
Perez2022_1045_T0            4  notF           Perez2022


In [6]:
SLE_metadata <- SLE_metadata %>%
  rownames_to_column(var = "sampleID")

# Mean activity accross celltypes

## Level 1

In [32]:
l1_cell_type_order <- names(cell_colors)
tf_order <- c("STAT1", "SP1")

In [26]:
toplot_perez_l1 <-
    perez2022_l1 %>%
    filter(!Level1 %in% c("Cycling_cells", "Progenitors", "Platelets"))

toplot_scgt00_l1 <-
    scgt00_l1 %>%
    filter(!Level1 %in% c("Cycling_cells", "Progenitors", "Platelets"))
toplot_level1 <- rbind(toplot_perez_l1, toplot_scgt00_l1)  %>%
  pivot_longer(cols = c(STAT1, SP1), names_to = "TF", values_to = "activity") %>%
  mutate(
    Level1 = factor(Level1, levels = l1_cell_type_order),
    TF = factor(TF, levels = tf_order)
  )

**Boxplots**

In [29]:
plot_l1 <- ggboxplot(toplot_level1, x = "Level1", y = "activity", fill = "Level1") +
    facet_grid(rows = vars(TF)#, scales = "free"
              ) +
    scale_fill_manual(values = cell_colors) +
    theme_bw() +
    ggtitle(label = "") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_l1,
    filename = "sle_TFactivity_level1.pdf",
    device = "pdf",
    width = 10,
    height = 5,
    path = base_path)

## Level 2

In [33]:
l2_cell_type_order <- names(cell_level2_colors)
tf_order <- c("STAT1", "SP1")

In [34]:
toplot_perez <-
    perez2022 %>%
    filter(!Level1 %in% c("Cycling_cells", "Progenitors", "Platelets"))

toplot_scgt00 <-
    scgt00 %>%
    filter(!Level1 %in% c("Cycling_cells", "Progenitors", "Platelets"))
toplot_level2 <- rbind(toplot_perez, toplot_scgt00)  %>%
  pivot_longer(cols = c(STAT1, SP1), names_to = "TF", values_to = "activity") %>%
  mutate(
    Level1 = factor(Level1, levels = l1_cell_type_order),
    Level2 = factor(Level2, levels = l2_cell_type_order),
    TF = factor(TF, levels = tf_order)
  )

**Boxplots**

In [39]:
plot_l2 <- ggboxplot(toplot_level2, x = "Level2", y = "activity", fill = "Level2") +
    facet_grid(rows = vars(TF), cols = vars(Level1), scales = "free_x"
              ) +
    scale_fill_manual(values = cell_level2_colors) +
    theme_bw() +
    ggtitle(label = "") +
    theme(
        axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_l2,
    filename = "sle_TFactivity_level2.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = base_path)

# SLEDAI vs Mean activity correlation

## Scatter plot

### SCGT00

In [13]:
head(scgt00)

,STAT1,SP1,studyID,Level1,Level2,sex,sampleID,disease,binned_age,SLEDAI_score,SLEDAI_category,Flare,Responder
,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<lgl>,<chr>
SCGT00_I014010.3P_T0_Mono_IFNresponse,0.4664260,1.9989017,SCGT00,Mono,Mono_IFNresponse,female,SCGT00_I014010.3P_T0,SLE,51-60,17.47,High,NA,NR
SCGT00_I014011.3P_T0_Mono_IFNresponse,0.4980574,1.5650074,SCGT00,Mono,Mono_IFNresponse,female,SCGT00_I014011.3P_T0,SLE,61-70,9.00,High,NA,R
SCGT00_I014012.3P_T0_Mono_IFNresponse,1.7604849,0.8499721,SCGT00,Mono,Mono_IFNresponse,female,SCGT00_I014012.3P_T0,SLE,18-30,12.00,High,NA,R
SCGT00_I014013.3P_T0_Mono_IFNresponse,1.6027005,-0.5147295,SCGT00,Mono,Mono_IFNresponse,female,SCGT00_I014013.3P_T0,SLE,18-30,10.00,High,NA,R
SCGT00_I014014.3P_T0_Mono_IFNresponse,1.6969669,-0.1240213,SCGT00,Mono,Mono_IFNresponse,female,SCGT00_I014014.3P_T0,SLE,41-50,8.00,Low,NA,NR
SCGT00_I014015.3P_T0_Mono_IFNresponse,1.6835376,0.5349421,SCGT00,Mono,Mono_IFNresponse,female,SCGT00_I014015.3P_T0,SLE,31-40,8.00,Low,NA,R


**Leve1 and Level2**

In [20]:
toplot_scgt00 <- scgt00 %>%
  filter(!Level1 %in% c("Cycling_cells", "Progenitors", "Platelets"))
toplot_scgt00 <- toplot_scgt00 %>%
  pivot_longer(cols = c(STAT1, SP1), names_to = "TF", values_to = "activity")

In [21]:
# Define the response palette
response_palette <- c("R" = "#70e000", "NR" = "#007200")

# Generate the scatterplot with facet_grid
scatter_scgt00 <- ggplot(toplot_scgt00, aes(x = SLEDAI_score, y = activity, fill = Level2, shape=Responder)) +
  geom_point(size = 3,  color = "black") +
  facet_grid(rows = vars(TF), cols = vars(Level1)) +
  scale_fill_manual(values = cell_level2_colors) +
  scale_shape_manual(values = c("R" = 21, "NR" = 24)) +
  theme(legend.position = "none") + 
  #guides(fill = guide_legend(title = "Level 2"), shape = guide_legend(title = "Response")) +
  labs(x = "SLEDAI Score", y = "Activity", title = "Scatterplot of SLEDAI Score vs Activity") 
  

ggsave(
    scatter_scgt00,
    filename = "scatter_scgt00.pdf",
    device = "pdf",
    width = 20,
    height = 10,
    path = "results/SLE_subgroups/")

ERROR: Error in grDevices::pdf(file = filename, ..., version = version): cannot open file 'results/SLE_subgroups//scatter_scgt00.pdf'


**Level1**

In [22]:
toplot_scgt00_l1 <- scgt00_l1 %>%
  filter(!Level1 %in% c("Cycling_cells", "Progenitors", "Platelets"))
toplot_scgt00_l1 <- toplot_scgt00_l1 %>%
  pivot_longer(cols = c(STAT1, SP1), names_to = "TF", values_to = "activity")

# Define the response palette
response_palette <- c("R" = "#70e000", "NR" = "#007200")

# Generate the scatterplot with facet_grid
scatter_scgt00_l1 <- ggplot(toplot_scgt00_l1, aes(x = SLEDAI_score, y = activity, fill = Level1, shape=Responder)) +
  geom_point(size = 3,  color = "black") +
  facet_grid(rows = vars(TF), cols = vars(Level1)) +
  scale_fill_manual(values = cell_colors) +
  scale_shape_manual(values = c("R" = 21, "NR" = 24)) +
  theme(legend.position = "none") + 
  #guides(fill = guide_legend(title = "Level 2"), shape = guide_legend(title = "Response")) +
  labs(x = "SLEDAI Score", y = "Activity", title = "Scatterplot of SLEDAI Score vs Activity") 
  

ggsave(
    scatter_scgt00_l1,
    filename = "scatter_scgt00_lev1.pdf",
    device = "pdf",
    width = 20,
    height = 10,
    path = "results/SLE_subgroups/")

ERROR: Error in grDevices::pdf(file = filename, ..., version = version): cannot open file 'results/SLE_subgroups//scatter_scgt00_lev1.pdf'


### Perez2022

There is no SLEDAI information for Flare patients, we can't do this comparison

# Boxplots individually per SLE subgroups

## Level 1

### Flare

In [10]:
toplot_flare_l1 <-
    perez2022_l1 %>%
    mutate(
        Flare = case_when(Flare == "not_F" ~ "notF",
        TRUE ~ as.character(Flare))
    ) %>%
    filter(
        Flare %in% c("notF", "F")
    )

flare_palette <- c(
  "F" = "#e76f51",
  "notF" = "#e9c46a"
)

**SP1**

In [11]:
plot_flare_sp1 <- ggboxplot(toplot_flare_l1, x = "Level1", y = "SP1", fill = "Flare") +
    #facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = flare_palette) +
    theme_bw() +
    ggtitle(label = "Flare SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_flare_sp1,
    filename = "flare_sp1_level1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [12]:
plot_flare_stat1 <- ggboxplot(toplot_flare_l1, x = "Level1", y = "STAT1", fill = "Flare") +
    #facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = flare_palette) +
    theme_bw() +
    ggtitle(label = "Flare STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_flare_stat1,
    filename = "flare_stat1_level1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

### Responder

In [14]:
response_palette <- c(
    "R" = "#70e000",
    "NR" = "#007200"
)

toplot_response <-
    scgt00_l1 %>%
    filter(
        Responder %in% c("R", "NR")
    )

**SP1**

In [15]:
plot_response_sp1 <- ggboxplot(toplot_response, x = "Level1", y = "SP1", fill = "Responder") +
    #facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = response_palette) +
    theme_bw() +
    ggtitle(label = "Response SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_response_sp1,
    filename = "response_sp1_level1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [16]:
plot_response_stat1 <- ggboxplot(toplot_response, x = "Level1", y = "STAT1", fill = "Responder") +
    #facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = response_palette) +
    theme_bw() +
    ggtitle(label = "Response STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_response_stat1,
    filename = "response_stat1_level1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

### SLEDAI

In [17]:
SLEDAI_palette <- c(
    "High" = "#103783",
    "Low" = "#9bafd9"
)

toplot_sledai_perez_l1 <-
    perez2022_l1 %>%
    filter(
        SLEDAI_category %in% c("High", "Low")
    )

toplot_sledai_scgt00_l1 <-
    scgt00_l1 %>%
    filter(
        SLEDAI_category %in% c("High", "Low")
    )

In [18]:
toplot_sledai <- rbind(toplot_sledai_perez_l1, toplot_sledai_scgt00_l1)

**SP1**

In [24]:
plot_sledai_sp1 <- ggboxplot(toplot_sledai, x = "Level1", y = "SP1", fill = "SLEDAI_category") +
    facet_grid(studyID ~ Level1, scales = "free_x") +
    scale_fill_manual(values = SLEDAI_palette) +
    theme_bw() +
    ggtitle(label = "SLEDAI SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_sledai_sp1,
    filename = "sledai_sp1_level1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [26]:
plot_sledai_stat1 <- ggboxplot(toplot_sledai, x = "Level1", y = "STAT1", fill = "SLEDAI_category") +
    facet_grid(studyID ~ Level1, scales = "free_x") +
    scale_fill_manual(values = SLEDAI_palette) +
    theme_bw() +
    ggtitle(label = "SLEDAI STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_sledai_stat1,
    filename = "sledai_stat1_level1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

## Level 2

### Flare

In [18]:
toplot_flare <-
    perez2022 %>%
    mutate(
        Flare = case_when(Flare == "not_F" ~ "notF",
        TRUE ~ as.character(Flare))
    ) %>%
    filter(
        Flare %in% c("notF", "F")
    )

flare_palette <- c(
  "F" = "#e76f51",
  "notF" = "#e9c46a"
)

**SP1**

In [26]:
plot_flare_sp1 <- ggboxplot(toplot_flare, x = "Level2", y = "SP1", fill = "Flare") +
    facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = flare_palette) +
    theme_bw() +
    ggtitle(label = "Flare SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_flare_sp1,
    filename = "flare_sp1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [27]:
plot_flare_stat1 <- ggboxplot(toplot_flare, x = "Level2", y = "STAT1", fill = "Flare") +
    facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = flare_palette) +
    theme_bw() +
    ggtitle(label = "Flare STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_flare_stat1,
    filename = "flare_stat1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

### Responder

In [29]:
response_palette <- c(
    "R" = "#70e000",
    "NR" = "#007200"
)

toplot_response <-
    scgt00 %>%
    filter(
        Responder %in% c("R", "NR")
    )

**SP1**

In [34]:
plot_flare_sp1 <- ggboxplot(toplot_response, x = "Level2", y = "SP1", fill = "Responder") +
    facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = response_palette) +
    theme_bw() +
    ggtitle(label = "Response SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_flare_sp1,
    filename = "response_sp1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [35]:
plot_flare_sp1 <- ggboxplot(toplot_response, x = "Level2", y = "STAT1", fill = "Responder") +
    facet_wrap(~Level1, scales = "free", ncol = 4, nrow = 2) +
    scale_fill_manual(values = response_palette) +
    theme_bw() +
    ggtitle(label = "Response STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_flare_sp1,
    filename = "response_stat1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

### SLEDAI

In [36]:
SLEDAI_palette <- c(
    "High" = "#103783",
    "Low" = "#9bafd9"
)

toplot_sledai_perez <-
    perez2022 %>%
    filter(
        SLEDAI_category %in% c("High", "Low")
    )

toplot_sledai_scgt00 <-
    scgt00 %>%
    filter(
        SLEDAI_category %in% c("High", "Low")
    )

In [38]:
toplot_sledai <- rbind(toplot_sledai_perez, toplot_sledai_scgt00)

**SP1**

In [43]:
plot_sledai_sp1 <- ggboxplot(toplot_sledai, x = "Level2", y = "SP1", fill = "SLEDAI_category") +
    facet_grid(studyID ~ Level1, scales = "free_x") +
    scale_fill_manual(values = SLEDAI_palette) +
    theme_bw() +
    ggtitle(label = "SLEDAI SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_sledai_sp1,
    filename = "sledai_sp1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [44]:
plot_sledai_stat1 <- ggboxplot(toplot_sledai, x = "Level2", y = "STAT1", fill = "SLEDAI_category") +
    facet_grid(studyID ~ Level1, scales = "free_x") +
    scale_fill_manual(values = SLEDAI_palette) +
    theme_bw() +
    ggtitle(label = "SLEDAI STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_sledai_stat1,
    filename = "sledai_stat1.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

#### Threshold = 4

In [70]:
SLEDAI_palette <- c(
    "High" = "#103783",
    "Low" = "#9bafd9"
)

toplot_sledai_perez <- perez2022 %>%
  mutate(SLEDAI_score = as.numeric(as.character(SLEDAI_score)),
         SLEDAI_category = ifelse(SLEDAI_score >= 4, "High", "Low")) %>%
  filter(SLEDAI_category %in% c("High", "Low"))


toplot_sledai_scgt00 <- scgt00 %>%
  mutate(SLEDAI_score = as.numeric(as.character(SLEDAI_score)),
         SLEDAI_category = ifelse(SLEDAI_score >= 4, "High", "Low")) %>%
  filter(SLEDAI_category %in% c("High", "Low"))


# Print the first few rows to verify the changes
print(table(toplot_sledai_perez$SLEDAI_category))
print(table(toplot_sledai_scgt00$SLEDAI_category))


High  Low 
 624 1232 

High 
 277 


In [71]:
toplot_sledai <- rbind(toplot_sledai_perez, toplot_sledai_scgt00)

**SP1**

In [72]:
plot_sledai_sp1 <- ggboxplot(toplot_sledai, x = "Level2", y = "SP1", fill = "SLEDAI_category") +
    facet_grid(studyID ~ Level1, scales = "free_x") +
    scale_fill_manual(values = SLEDAI_palette) +
    theme_bw() +
    ggtitle(label = "SLEDAI SP1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_sledai_sp1,
    filename = "sledai_sp1_thr4.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")

**STAT1**

In [73]:
plot_sledai_stat1 <- ggboxplot(toplot_sledai, x = "Level2", y = "STAT1", fill = "SLEDAI_category") +
    facet_grid(studyID ~ Level1, scales = "free_x") +
    scale_fill_manual(values = SLEDAI_palette) +
    theme_bw() +
    ggtitle(label = "SLEDAI STAT1") +
    theme(
        axis.text.x = element_text(angle = 25, vjust = 0.5, hjust=0.5)
    )

ggsave(
    plot_sledai_stat1,
    filename = "sledai_stat1_thr4.pdf",
    device = "pdf",
    width = 20,
    height = 8,
    path = "results/SLE_subgroups/")